### 필요한 패키지 설치

In [ ]:
# pip install beautifulsoup4 langchain langchain-text-splitters langchain-community faiss-cpu langchain-core langchain-openai openai

### 필요한 라이브러리 설치

In [13]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

### 환경설정

In [2]:
import os

os.environ['OPENAI_API_KEY'] = "api_key"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "project_name"
os.environ["LANGCHAIN_API_KEY"] = "api_key"

### WebBaseLoader 이용해서 웹페이지 내용 추출하기

In [7]:
loader = WebBaseLoader(
    # 소괄호 안에 ','를 넣는 이유는 '단일 요소만 있다'는 걸 알리기 위해서.
    web_paths = ("https://n.news.naver.com/article/437/0000378416",),
    # 'bs_kwargs'는 BeautifulSoup 키워드라는 뜻
    # SoupStrainer() 첫번째 인자는 문자열만 넣어도 태그로 인식
    bs_kwargs = dict(
        parse_only= bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]}
        )
    )
)

docs = loader.load()
print(f"문서의 수 : {len(docs)}")

문서의 수 : 1


### RecursiveCharacterTextSplitter 이용해서 문서 내용 쪼개기

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

3

### vectorstore에 쪼갠 문서 담기 (임배딩)

In [22]:
# FAISS는 페이스북에서 만든 백터 데이터베이스 라이브러리
# FAISS (Facebook AI Similarity Search)
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

### PromptTemplate 이용해서 프롬프트 생성

In [24]:
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

### llm 모델 생성 후 기본체인 구성

In [25]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# RunnablePassthrough에는 rag_chain이 실행 될 때 들어오는 질문이 들어간다.
# 질문이 들어 오면, retriever가 질문에 적절한 내용은 쪼개 놓은 문서에서 찾는다.
# 찾은 내용을 context에 넣는다.
# context, question 내용이 적용된 prompt는 llm으로 전달한다.
# llm은 StrOutputParser()에 의해 질문에 대한 답변을 생성한다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### 실시간 출력(스트리밍 출력) 하기 위한 함수 정의

In [28]:
class StreamChain:
    # 자동으로 생성되는 내장함수이지만, 일부러 만든 이유는 chain을 받기 위해서.
    def __init__(self, chain):
        self.chain = chain

    def stream(self, query):
        # 사용자 질문이 'query'로 들어간다. 
        # self.chain(기본체인)에 따라 답변이 생성되고, 해당 답변은 'response'에 담긴다.
        response = self.chain.stream(query)
        # 아무 값이 없는 빈 공간을 만들어준다. 해당 공간은 답변이 담길 공간이다.
        complete_response = ""
        # 생성된 답변을 반복문을 이용해 형태소 단위로 쪼개어 준다.
        for token in response:
            # end=""는 줄바꿈 없이 계속 실행하라는 코드다.
            # flush=True는 '실시간 출력해라'라는 의미를 가진 코드다.
            print(token, end="", flush=True)
            # 실시간으로 출력된 형태소를 합쳐주는 역할을 한다.
            complete_response += token
        return complete_response

chain = StreamChain(rag_chain)

### 체인과 함수 이용해 답변 출력

> [langsmith 이용해서 답변 출력 과정 보기](https://smith.langchain.com/)

In [29]:
answer = chain.stream("부영그룹의 출산 장려 정책에 대해 설명해주세요.")

부영그룹의 출산 장려 정책은 2021년 이후 태어난 직원 자녀에게 1억원을 지원하는 것입니다. 이 정책은 총 70억원 규모로, 연년생이나 쌍둥이 자녀가 있을 경우 총 2억원을 받을 수 있습니다. 또한, 셋째 자녀를 낳는 경우에는 국민주택을 제공하겠다는 계획도 밝혔습니다. 이 외에도 출산장려금에 대한 세금 면세를 정부에 제안하기도 했습니다.

In [30]:
# rag와 프롬프트를 이용했기 때문에 할루시네이션이 발생하지 않았다
answer = chain.stream("부영그룹의 임직원 숫자는 몇명인가요?")

주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.